### Base chatbot

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDeepInfra

template = """Question: {question}
Answer: Let's think step by step."""

DEEPINFRA_CHAT_MODEL = os.getenv("DEEPINFRA_CHAT_MODEL")
DEEPINFRA_API_TOKEN = os.getenv("DEEPINFRA_API_TOKEN")
prompt = ChatPromptTemplate.from_template(template=template)
model = ChatDeepInfra(model=DEEPINFRA_CHAT_MODEL, temperature=0, deepinfra_api_token=DEEPINFRA_API_TOKEN)

chain = prompt | model
response = chain.invoke({"question": "What is 2+2"})
print(response.content)

/Users/michaeleco/Documents/Github/zenith-ai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Step 1:** Identify the numbers to add.  
- The first number is **2**.  
- The second number is **2**.

**Step 2:** Perform the addition.  
- Start with the first number: 2.  
- Add the second number: 2 + 2.

**Step 3:** Calculate the sum.  
- 2 + 2 = 4.

**Answer:** 2 + 2 equals **4**.


### Retrieve agent

In [2]:
from qdrant_client import QdrantClient

if os.getenv("IS_DOCKER"):
    qdrant_host = "localhost"
else:
    qdrant_host = os.getenv("QDRANT_HOST")
    
qdrant_url = f"http://{qdrant_host}:{os.getenv('QDRANT_PORT')}"
client = QdrantClient(url=qdrant_url)

In [3]:
from langchain_community.embeddings import DeepInfraEmbeddings

user_query = "berapa harga face peeling?"
deepinfra_embedding = DeepInfraEmbeddings(
    model_id=os.getenv("DEEPINFRA_EMBEDDING_MODEL"),
    embed_instruction="",
    query_instruction=""
)

embedding = deepinfra_embedding.embed_query(user_query)

In [4]:
result = client.query_points(
    collection_name=os.getenv("QDRANT_COLLECTION"),
    query=embedding,
    limit=50
)

ResponseHandlingException: [Errno 61] Connection refused

In [ ]:
template = """Anda adalah asisten AI untuk klinik kecantikan (ERHA/Dermies).
Gunakan konteks berikut untuk menjawab pertanyaan pengguna dengan akurat dan profesional.

Aturan:
1. Jika konteks tidak memuat jawaban, katakan "Maaf, informasi tersebut tidak tersedia dalam dokumen kami."
2. Jawablah dalam Bahasa Indonesia yang sopan.
3. Usahakan jawaban singkat dan padat (maksimal 3-4 kalimat).
4. Jika menyebutkan harga, pastikan sesuai dengan yang tertulis di dokumen.

Pertanyaan: {question} 

Konteks: 
{context} 

Jawaban:
"""

prompt = ChatPromptTemplate.from_template(template=template)
chain = prompt | model
response = chain.invoke({"question": user_query, "context": result})
print(response)


content='Harga face peeling (Derma Peeling for Face) mulai dari **Rp525.000**.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 18375, 'total_tokens': 18570, 'completion_tokens': 195, 'estimated_cost': 0.00057855, 'prompt_tokens_details': None}, 'model': 'openai/gpt-oss-20b', 'finish_reason': 'stop'} id='lc_run--722087b7-cb0b-4118-bbcb-530abeceb2f1-0'


### Using reranker

In [ ]:
import mlflow

tracking_uri = "http://127.0.0.1:5050"

mlflow.set_tracking_uri(tracking_uri)
client = mlflow.tracking.MlflowClient(tracking_uri=tracking_uri, registry_uri=tracking_uri)

versions = client.get_latest_versions("XGBoostReranker", stages=["Staging"])
latest_version = versions[0].version
MODEL_URI = f"models:/XGBoostReranker/{latest_version}"
reranker_model = mlflow.xgboost.load_model(MODEL_URI)

/var/folders/17/504r7zwd36jcb_pwxly46r940000gn/T/ipykernel_13064/2717055463.py:8: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions("XGBoostReranker", stages=["Staging"])


In [ ]:
import pandas as pd

data = []
for point in result.points:
    payload = point.payload
    data.append({
        "query_text": user_query,
        "doc_id": point.id,
        "full_text": payload.get('full_text', ''),
        "h1": payload.get('h1', ''),
        "qdrant_score": point.score,
        "payload": payload
    })

df_candidates = pd.DataFrame(data)
df_candidates.head()

,query_text,doc_id,full_text,h1,qdrant_score,payload
0,berapa harga face peeling?,abf91c39-2163-4be6-a75d-e83c2aa83088,Derma Peeling for Face\n\nPeeling wajah yang b...,724126666-ERHA-Ultimate-Pricelist-24.pdf,0.560869,{'full_text': 'Derma Peeling for Face Peeling...
1,berapa harga face peeling?,2f881a88-ca5a-4868-9e06-75c8c594ef83,ACNE CENTER\n\nSINGLE TREATMENT\n\nAcne Peelin...,724126666-ERHA-Ultimate-Pricelist-24.pdf,0.543833,{'full_text': 'ACNE CENTER SINGLE TREATMENT ...
2,berapa harga face peeling?,746f4753-8319-439f-a5e2-0e34e252e562,"MANFAAT\n\nKulit wajah menjadi lebih halus, se...",724126666-ERHA-Ultimate-Pricelist-24.pdf,0.542393,{'full_text': 'MANFAAT Kulit wajah menjadi le...
3,berapa harga face peeling?,81022a71-697a-448c-ad54-cab19b030182,DERMIAS MAX\n\nSingle Advanced Prime Purifying...,661627558-Katalog-Dermies-Max-by-Erha-Hiress.pdf,0.536641,{'full_text': 'DERMIAS MAX Single Advanced Pr...
4,berapa harga face peeling?,c17893de-5ddb-4035-9dcd-60b117484219,e Jelly Ice Facial for Acne\n\nStart From Rp 1...,661627558-Katalog-Dermies-Max-by-Erha-Hiress.pdf,0.514398,{'full_text': 'e Jelly Ice Facial for Acne St...


In [ ]:
from rapidfuzz import fuzz

features = pd.DataFrame()
        
# Normalize text
df_candidates['q_lower'] = df_candidates['query_text'].astype(str).str.lower()
df_candidates['doc_lower'] = df_candidates['full_text'].astype(str).str.lower()
df_candidates['h1_lower'] = df_candidates['h1'].astype(str).str.lower()

# 1. Vector Score
features['qdrant_score'] = df_candidates['qdrant_score']

# 2. Lengths
features['doc_len'] = df_candidates['doc_lower'].apply(len)
features['query_len'] = df_candidates['q_lower'].apply(len)

# 3. Word Overlap
def word_overlap(row):
    q_tokens = set(row['q_lower'].split())
    d_tokens = set(row['doc_lower'].split())
    if not q_tokens: return 0.0
    return len(q_tokens.intersection(d_tokens)) / len(q_tokens)
features['word_overlap'] = df_candidates.apply(word_overlap, axis=1)

# 4. Header Match
features['match_in_h1'] = df_candidates.apply(
    lambda x: fuzz.partial_ratio(x['q_lower'], x['h1_lower']), axis=1
)

# 5. Fuzzy Match
features['fuzzy_ratio'] = df_candidates.apply(
    lambda x: fuzz.ratio(x['q_lower'], x['doc_lower'][:500]), axis=1
)

# 6. Price Heuristic
def price_relevance(row):
    is_price_query = any(w in row['q_lower'] for w in ['harga', 'biaya', 'price', 'rp'])
    has_price_info = 'rp' in row['doc_lower'] or 'rp.' in row['doc_lower']
    return 1 if (is_price_query and has_price_info) else 0
features['is_price_match'] = df_candidates.apply(price_relevance, axis=1)
features.head()

,qdrant_score,doc_len,query_len,word_overlap,match_in_h1,fuzzy_ratio,is_price_match
0,0.560869,1065,26,0.25,38.461538,8.745247,1
1,0.543833,874,26,0.25,38.461538,9.505703,1
2,0.542393,843,26,0.25,38.461538,9.505703,1
3,0.536641,1000,26,0.25,36.363636,8.745247,1
4,0.514398,1182,26,0.00,36.363636,8.745247,1


In [ ]:
scores = reranker_model.predict(features)
scores

array([-0.26256603, -1.0957012 , -0.9364217 , -0.39228195, -1.1433628 ,
       -0.77391857, -1.2232178 , -1.6909393 , -1.0393666 , -1.1394922 ,
       -1.5020508 , -0.77391857, -2.2768548 , -1.4432393 , -1.4755443 ,
       -1.1433628 , -2.477151  , -0.77391857, -0.70627123, -2.6108296 ,
       -2.477151  , -1.1433628 , -1.6909393 , -1.1394922 , -2.4198909 ,
       -1.1433628 , -1.6909393 , -1.1394922 , -1.1394922 , -0.6699217 ,
       -1.6909393 , -2.6108296 , -1.7123667 , -1.6909393 , -1.6909393 ,
       -2.4198909 , -1.0604664 , -0.77391857, -1.0393666 , -1.7986232 ,
       -1.098164  , -2.2768548 , -1.0757159 , -1.0604664 , -1.1433628 ,
       -1.7123667 , -2.6108296 , -1.6909393 , -1.2232178 , -1.6909393 ],
      dtype=float32)

In [ ]:
df_candidates['rerank_score'] = scores
df_candidates = df_candidates.sort_values(by='rerank_score', ascending=False)
df_candidates.head()
reranked_results = df_candidates.to_dict(orient='records')
top_docs = reranked_results[:5]
top_docs

In [ ]:
template = """Anda adalah asisten AI untuk klinik kecantikan (ERHA/Dermies).
Gunakan konteks berikut untuk menjawab pertanyaan pengguna dengan akurat dan profesional.

Aturan:
1. Jika konteks tidak memuat jawaban, katakan "Maaf, informasi tersebut tidak tersedia dalam dokumen kami."
2. Jawablah dalam Bahasa Indonesia yang sopan.
3. Usahakan jawaban singkat dan padat (maksimal 3-4 kalimat).
4. Jika menyebutkan harga, pastikan sesuai dengan yang tertulis di dokumen.

Pertanyaan: {question} 

Konteks: 
{context} 

Jawaban:
"""

prompt = ChatPromptTemplate.from_template(template=template)
model = ChatDeepInfra(model=DEEPINFRA_CHAT_MODEL, temperature=0, deepinfra_api_token=DEEPINFRA_API_TOKEN)
chain = prompt | model

print("Generating Answer")
response = chain.invoke({"question": user_query, "context": top_docs})
response.content

Generating Answer


'Harga face peeling (Derma Peeling for Face) mulai dari **Rp525.000**.'